# Création de prompts pour le nommage et la description des clusters

In [2]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [3]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


In [4]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceOther\", \"ses_immigrant\", \"ses_regionOntario\", \"ses_regionWest\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"ses_age\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 50.3%, Immigrants: 32.4%\nÂge: 41.4 ans\nRevenus:\n  Revenu Bas: 14.5 %\n  Revenu Mid: 68.3 %\n  Revenu High: 17.2 %\nLangues:\n  En: 89%, Fr: 2.1%, Autres: 9%\nEducBHS: 24.1%\nEducPostHS: 29%\nEducUniv: 46.9%\nEthnie (Blanc): 67.6%\nOrientation (Hétéro): 93.1%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceCPC\", \"ses_educUniv\", \"ses_incomeHigh\", \"ses_regionOntario\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_educBHS\", \"ses_languageFrench\", \"ses_regionQuebec\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 50.8%, Immigrants: 23.3%\nÂge: 47.7 ans\nRevenus:\n  Revenu Bas: 12.5 %\n  Revenu Mid: 60.8 %\n  Revenu High: 26.7 %\nLangues:\n  En: 89.2%, Fr: 1.7%, Autres: 9.2%\nEducBHS: 11.7%\nEducPostHS: 30.8%\nEducUniv: 57.5%\nEthnie (Blanc): 73.3%\nOrientation (Hétéro): 94.2%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 3 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nses_regionAtlantic\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"ses_educUniv\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 59.5%, Immigrants: 18.9%\nÂge: 53 ans\nRevenus:\n  Revenu Bas: 13.5 %\n  Revenu Mid: 81.1 %\n  Revenu High: 5.4 %\nLangues:\n  En: 100%, Fr: 0%, Autres: 0%\nEducBHS: 29.7%\nEducPostHS: 40.5%\nEducUniv: 29.7%\nEthnie (Blanc): 94.6%\nOrientation (Hétéro): 78.4%\n"

$`4`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 4 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceBQ\", \"ses_age\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_immigrant\", \"ses_incomeHigh\", \"ses_regionOntario\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 59.8%, Immigrants: 4.1%\nÂge: 64.9 ans\nRevenus:\n  Revenu Bas: 12.3 %\n  Revenu Mid: 82 %\n  Revenu High: 5.7 %\nLangues:\n  En: 3.3%, Fr: 96.7%, Autres: 0%\nEducBHS: 17.2%\nEducPostHS: 41%\nEducUniv: 41.8%\nEthnie (Blanc): 95.9%\nOrientation (Hétéro): 91%\n"

$`5`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 5 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceOther\", \"ses_educBHS\", \"ses_incomeLow\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\",

In [5]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [6]:
reponses_openai

$`1`
[1] "Pour une personne habitant au Canada correspondant à ce profil, je propose le prénom \"Liam\".\n\n**Description du persona :**\n\n**Prénom :** Liam\n\n**Âge :** 41 ans\n\n**Origine :** Immigrant (32.4% des membres du cluster)\n\n**Lieu de résidence :** Probablement en Ontario ou dans l'Ouest canadien, car il y a une surreprésentation dans ces régions et une sous-représentation au Québec.\n\n**Langue principale :** Anglais (89% d'anglophones)\n\n**Éducation :** Très éduqué, avec une forte probabilité d'avoir un diplôme universitaire (46.9% ont une éducation universitaire).\n\n**Revenu :** Fait probablement partie de la classe moyenne (68.3%), mais il y a aussi une bonne part qui a un revenu élevé (17.2%).\n\n**Ethnicité :** Probablement blanc, mais il y a une diversité ethnique notable.\n\n**Orientation sexuelle :** Hétérosexuel (93.1%)\n\n**Intérêts politiques :** Moins enclin à voter pour les partis traditionnels tels que le Parti libéral, le Parti conservateur ou le NPD. Il est aussi moins susceptible de voter pour le Bloc Québécois et a une tendance plus élevée à choisir d'autres options politiques.\n\n**Participation politique :** Moins susceptible de voter en général, avec une faible solidité dans ses choix politiques.\n\n**Culture :** Vie dans un environnement multiculturel, ce qui est typique pour un immigrant au Canada, surtout dans des régions comme l'Ontario.\n\n**Carrière :** Avec un haut niveau d'éducation et un revenu moyen à élevé, il est probable qu'il occupe un poste professionnel ou managérial.\n\n**Personnalité :** Ouvert et tolérant, avec des expériences diverses en raison de son statut d'immigrant et de son cadre multiculturel."

$`2`
[1] "Pour une personne vivant au Canada avec le profil décrit, un prénom qui pourrait convenir est \"David\". \n\n**Persona 2: David**\n\n- **Âge**: 47 ans  \n- **Sexe**: Homme\n- **Statut d'immigration**: 23.3% de chances d'être immigrant\n- **Revenu**: \n  - Bas: 12.5%\n  - Moyen: 60.8%\n  - Élevé: 26.7%\n- **Région**: Probablement en Ontario, étant donné la forte moyenne dans cette région et une faible proportion au Québec ou dans l'Ouest canadien.\n- **Langue**: Principalement anglophone (89.2%)\n- **Éducation**: Universitaire (57.5%)\n- **Ethnie**: Très probablement blanche (73.3%)\n- **Orientation sexuelle**: Hétérosexuelle (94.2%)\n- **Préférences politiques**: \n  - Fortement enclin à soutenir le Parti conservateur du Canada (CPC)\n  - Faible probabilité de soutenir le Bloc québécois, le Parti libéral du Canada, le Nouveau Parti démocratique ou d'autres partis.\n\nDavid est un homme instruit et implique dans la vie politique, ayant une forte tendance à voter pour des candidats conservateurs. Il vit probablement en Ontario, est bien intégré économiquement et socialement, et valorise la stabilité et la sécurité."

$`3`
[1] "Pour une personne habitant au Canada, particulièrement dans la région Atlantique, un prénom approprié pourrait être \"James\". Voici une brève description du persona 3 :\n\n**James, 53 ans, réside dans la région Atlantique du Canada.**\n\nJames est représentatif de son groupe démographique dans cette région. Il est anglophone et fait partie de la majorité blanche. Bien qu'il ne soit pas un immigrant, il cohabite avec une certaine diversité, sachant qu'environ 18,9% de son groupe sont des immigrants. James a terminé des études secondaires et a pu poursuivre ses études au niveau post-secondaire, mais n'a pas nécessairement acquis un diplôme universitaire.\n\nProfessionnellement, James se situe dans la tranche de revenu moyen, ce qui correspond à 81,1% de son groupe, avec une minorité dans les tranches de revenu bas et élevé. Il ne s'intéresse pas particulièrement au Bloc Québécois, et le français n'est pas une langue qu'il utilise ou maîtrise.\n\nJames est hétérosexuel et partage des valeurs et des intérêts typiques de sa communauté dans la région Atlantique. Il aime probablement les activités en plein air que la région offre, ainsi que les év

In [7]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [8]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_voteChoiceOther"", ""ses_immigrant"", ""ses_regionOntario"", ""ses_regionWest"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_solidity"", ""dv_turnout"", ""dv_voteChoiceBQ"", ""dv_voteChoiceCPC"", ""dv_voteChoiceLPC"", ""dv_voteChoiceNDP"", ""ses_age"", ""ses_languageFrench"", ""ses_regionQuebec"") Ce cluster est composé de : Hommes: 50.3%, Immigrants: 32.4% Âge: 41.4 ans Revenus: Revenu Bas: 14.5 % Revenu Mid: 68.3 % Revenu High: 17.2 % Langues: En: 89%, Fr: 2.1%, Autres: 9% EducBHS: 24.1% EducPostHS: 29% EducUniv: 46.9% Ethnie (Blanc): 67.6% Orientation (Hétéro): 93.1%","Pour une personne habitant au Canada correspondant à ce profil, je propose le prénom ""Liam"". **Description du persona :** **Prénom :** Liam **Âge :** 41 ans **Origine :** Immigrant (32.4% des membres du cluster) **Lieu de résidence :** Probablement en Ontario ou dans l'Ouest canadien, car il y a une surreprésentation dans ces régions et une sous-représentation au Québec. **Langue principale :** Anglais (89% d'anglophones) **Éducation :** Très éduqué, avec une forte probabilité d'avoir un diplôme universitaire (46.9% ont une éducation universitaire). **Revenu :** Fait probablement partie de la classe moyenne (68.3%), mais il y a aussi une bonne part qui a un revenu élevé (17.2%). **Ethnicité :** Probablement blanc, mais il y a une diversité ethnique notable. **Orientation sexuelle :** Hétérosexuel (93.1%) **Intérêts politiques :** Moins enclin à voter pour les partis traditionnels tels que le Parti libéral, le Parti conservateur ou le NPD. Il est aussi moins susceptible de voter pour le Bloc Québécois et a une tendance plus élevée à choisir d'autres options politiques. **Participation politique :** Moins susceptible de voter en général, avec une faible solidité dans ses choix politiques. **Culture :** Vie dans un environnement multiculturel, ce qui est typique pour un immigrant au Canada, surtout dans des régions comme l'Ontario. **Carrière :** Avec un haut niveau d'éducation et un revenu moyen à élevé, il est probable qu'il occupe un poste professionnel ou managérial. **Personnalité :** Ouvert et tolérant, avec des expériences diverses en raison de son statut d'immigrant et de son cadre multiculturel."
2,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_solidity"", ""dv_turnout"", ""dv_voteChoiceCPC"", ""ses_educUniv"", ""ses_incomeHigh"", ""ses_regionOntario"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""dv_voteChoiceLPC"", ""dv_voteChoiceNDP"", ""dv_voteChoiceOther"", ""ses_educBHS"", ""ses_languageFrench"", ""ses_regionQuebec"", ""ses_regionWest"") Ce cluster est composé de : Hommes: 50.8%, Immigrants: 23.3% Âge: 47.7 ans Revenus: Revenu Bas: 12.5 % Revenu Mid: 60.8 % Revenu High: 26.7 % Langues: En: 89.2%, Fr: 1.7%, Autres: 9.2% EducBHS: 11.7% EducPostHS: 30.8% EducUniv: 57.5% Ethnie (Blanc): 73.3% Orientation (Hétéro): 94.2%","Pour une personne vivant au Canada avec le profil décrit, un prénom qui pourrait convenir est ""David"". **Persona 2: David** - **Âge**: 47 ans - **Sexe**: Homme - **Statut d'immigration**: 23.3% de chances d'être immigrant - **Revenu**: - Bas: 12.5% - Moyen: 60.8% - Élevé: 26.7% - **Région**: Probablement en Ontario, étant donné la forte moyenne dans cette région et une faible proportion au Québec ou dans l'Ouest canadien. - **Langue**: Principalement anglophone (89.2%) - **Éducation**: Universitaire (57.5%) - **Ethnie**: Très probablement blanche (73.3%) - **Orientation sexue

In [9]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [10]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")